In [ ]:
# Preparation for programming
# Make sure to execute this cell first!
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')                  # do not show warnings
from obspy import *
from obspy import read
from obspy.core import read, UTCDateTime
from obspy.clients.fdsn import Client
import matplotlib.pylab as plt
import os
from os import path
import pandas as pd
from datetime import datetime 
from sumatra_loop import sumatra_loop
import sys
import numpy as np
from obspy.signal.invsim import cosine_taper

# 
# Applied Seismology, GEOS 626, University of Alaska Fairbanks
# Python coding by Nealey Sims
# Template script for analyzing modes spectra for sumatra
#
# Downloads BH* and LH* data needed for the Sumatra B assignment
# 

# Plotting parameter
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['lines.linewidth'] = 1

In [ ]:
# Setting up parameters for getting waveforms
client = Client("IRIS")
t = UTCDateTime("2004-12-26T00:58:53.0")   # origin time of Sumatra earthquake
LHstarttime = t-(10*24*60*60)                 # 10 days before the the origin time
LHendtime = t+(10*24*60*60)                   # 10 days after the the origin time
BHstarttime = t-(1*60*60)                     # 1 hour before origin time
BHendtime = t+(4*60*60)                       # 4 hours after orgin time


# USER CHANGE THIS
Dload=False                                   # Run True once to download data

# path to waveform directory
LHdirectory = "./datawf/sumatraLH"
if not path.exists(LHdirectory):  # If data directory doesn't exist, it will create one
        os.makedirs(LHdirectory)
BHdirectory = "./datawf/sumatraBH"
if not path.exists(BHdirectory):  # If data directory doesn't exist, it will create one
        os.makedirs(BHdirectory)

In [ ]:
# Measure the download time
start = datetime.now()
stanames=[]
ntwk=[]
chans=[]
locs=[]
stimes=[]
etimes=[]
inventory = client.get_stations(network="G,II,IU",starttime=LHstarttime, endtime =LHendtime,  
                    channel="BH*,LH*", level="response")

inventory.write("sumatra_inv.txt",format="STATIONTXT")

if Dload==True:
    
    
    for net in inventory:     
        for sta in net:
            # Use inventory to save station locations
            for cha in sta:
                stanames.append(sta.code)
                ntwk.append(net.code)
                chans.append(cha.code) 
                locs.append(cha.location_code)
                if str(cha.code[0]) == "B":
                    stimes.append(BHstarttime)
                    etimes.append(BHendtime)
                else:
                    stimes.append(LHstarttime)
                    etimes.append(LHendtime)
            
    # Download and save the raw waveform in a new directory

    for i in range (len(stanames)):
        st=[]
        try:
            st = client.get_waveforms(ntwk[i], stanames[i], locs[i], chans[i],
                                  stimes[i], etimes[i], attach_response=True)     # get the waveform
        except:
            print("no data for request",stanames[i],ntwk[i],chans[i])
            pass
        if len(st) != 0:
            if chans[i][0]=="L":
                try:
                    st.resample(1.0)                              # resample the sampling rate
                    st.merge(method=1, fill_value=0)              # merge the traces
                except:
                    print("couldn't merge traces")
                    pass
                # write and save the stream into SAC format
                st.write(path.join(LHdirectory, str(stanames[i])+str(locs[i])+str(chans[i])), format = 'SAC')
            else:
                st.write(path.join(BHdirectory, str(stanames[i])+str(locs[i])+str(chans[i])), format = 'SAC')
            print (st)
# Print download time
d = datetime.now() - start
print (d, " s")


In [ ]:
# path of the fft directory 
directoryfft = "./datawf/sumatra_fft"
if not path.exists(directoryfft):  # If data directory doesn't exist, it will create one
    os.makedirs(directoryfft) 
drec = os.listdir(directoryfft) 
 
# Checking to see if fft has been calculated and saved 
if len(drec) < 3: 
    print("Empty directory, running sumatra_loop to save fft")
    LHZst = read(LHdirectory+"/*Z",header=None)
    w=sumatra_loop(LHZst)
    all_amps = read(directoryfft+"/*amps", header = None)
    all_f = np.load(directoryfft+'/all_fft_freq.npy')
    all_phs = np.load(directoryfft+'/all_fft_phase.npy')
else:
    LHZst = read(LHdirectory+"/*Z",header=None)
    print("sumatra_directory exists and is not empty")
    all_amps = read(directoryfft+"/*amps", header = None)
    all_f = np.load(directoryfft+'/all_fft_freq.npy')
    all_phs = np.load(directoryfft+'/all_fft_phase.npy')


In [ ]:
ind,ind_pdf,stas,chans,nets,tag,ikeep = np.genfromtxt('sumatra_modes.txt', unpack=True, dtype=str)
ksta=[]
scut=[]
pickall=[]
w=LHZst.copy()
for k in range(len(stas)):
    if int(ikeep[k])==1:
        ksta.append(stas[k])
        pickall.append(int(ind_pdf[k]))
    else:
        scut.append(k)
print('%i/%i bad records that will not be used:' %(len(scut),len(ind)))

# plot the 30 time series that were manually cut from the analysis
ii=0

while ii < len(ind):   # loop over all 169 time series
    if ii in scut:  
        stag = str(stas[ii])+ '_'+ str(chans[ii])+ '_'+ str(nets[ii])
        stdur = ('duration = %.2f days' % (len(w[ii])/86400))
        stit = str(stag) +', ' +str(stdur)
        w[ii].data = np.where(w[ii].data ==0.0000, np.nan, w[ii].data)
        print('%i/%i %s'%(ii,len(ind),stag))
        fig=plt.figure()
        plt.plot(w[ii].data, 'b')
        plt.xlabel('seconds')
        plt.ylabel('nm/sec')
        plt.xlim(0,20*86400)
        plt.ticklabel_format(axis="both", style="sci", scilimits=(0,0))
        plt.title(stit)
    ii+=1
print('break')   
sys.exit()  ### comment out when proceeding to next block        

In [ ]:
from scipy import interpolate
directory_fft = "./datawf/sumatra_fft"
# USER: PICK A SET TO PLOT AND SAVE FOR ANALYSIS
# note: these are the same values as the 2nd column sumatra_modes.txt, use those values to index

ipick=[0,1,2,19]
#ipick = pickall;    # view all 129 not-cut stations
stkeep=[]
stan=[]
a_freqs=[]
st_all=[]        

# Stack signal
for i in ipick:
    st_fft=all_amps[i].data
    #C = st_fft*np.exp(1j*all_phs[i]) # Fourier Transform C(w)
    #stkeep.append(C)
    st_all.append(st_fft)
    stan.append(ksta[i])
    a_freqs.append(all_f[i])   

In [ ]:
# Plot the full spectrum of the signal
for i in range(len(ipick)):
    indarray=st_all[i]
    indend=int(len(a_freqs[i])/(a_freqs[i].max()*1000))
    slicedArray = indarray[int(0.2*(a_freqs[i].max()*1000)):5000]
    plt.plot(a_freqs[i]*1000, abs(st_all[i]), color='b')
    plt.title('Amplitude Spectrum of %7s (Zoomed in)' % (stan[i]))
    plt.xlabel('Frequency [mHz]')
    plt.ylabel('Amplitude [counts]')
    plt.xlim (0.2, 1)
    plt.ylim (0, slicedArray.max()+5)
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.show()


In [ ]:
# Example of stacking spectra

def w2fstack(freq,amp,f1,f2,n):
    # Function used to stack waveforms given frequency and amplitude arrays
    nw = len(freq)
    f = np.linspace(f1,f2,n)
    A = np.zeros((n,nw))
    for ii in range (nw):
        f0 = freq[ii]  # Hz
        A0 = amp[ii]
        A[:,ii] = np.interp(f,f0,A0)    
    Astack = np.sum(A,1)
    return Astack,f,A

f1 = (0.2*1E-3)
f2 = (10*1E-3)
n = int(1E5)

Astack,fplot,A = w2fstack(a_freqs,st_all,f1,f2,n)

plt.plot(fplot*1E3,Astack,'b') 
plt.xlabel('frequency [mHz]')
plt.ylabel('stacked amplitude spectrum')
plt.show()

# note: matrix A contains all spectra
plt.plot(fplot,A)
plt.xlabel('frequency [mHz]')
plt.ylabel('stacked amplitude spectrum')
plt.show()

In [ ]:
# START MODES PROBLEM HERE


#==========================================================================
